# annotating_clinical_trial_results.ipynb
The goal of this notebook is to extract additional features from clinical trial result press releases for further analysis / modeling. 

In [ ]:
# Load Libraries
import matplotlib.pyplot as plt
from collections import Counter
import sqlite3
import json
import pandas as pd
from datetime import datetime, timedelta
from openai import OpenAI

# Helper Functions
def make_json_llm_call(client, prompt):
    import json
    open_ai_model = "gpt-4o-mini"
    chat_completion = client.chat.completions.create(
            messages=[
                {"role": "user", "content": f"{prompt}"},
            ],
            model=open_ai_model,
            response_format={ "type": "json_object" }

        )
    json_str = chat_completion.choices[0].message.content
    annotated_data = json.loads(json_str)
    return annotated_data


# fetch the mean close data of the 2 days before the event
def get_two_days_before_event(financial_data, ticker, date_str):
    # Get the DataFrame for the specific ticker
    df = financial_data[ticker].copy()
    df.index = pd.to_datetime(df.index)

    # Convert the input string to a datetime object
    event_date = datetime.strptime(date_str, "%m_%d_%Y")
    
    # Get all dates before the event
    prior_dates = df.index[df.index < event_date]
    
    # Get the two most recent dates before the event
    recent_dates = prior_dates.sort_values()[-2:]
    
    if len(recent_dates) < 2:
        return f"Not enough data before {event_date.date()} for ticker {ticker}"
    
    return df.loc[recent_dates]

def get_data_x_days_after_event(financial_data, ticker, date_str, range):
    # Get the DataFrame for the specific ticker
    df = financial_data[ticker].copy()
    df.index = pd.to_datetime(df.index)

    # Convert the input string to a datetime object
    event_date = datetime.strptime(date_str, "%m_%d_%Y")
    
    # Get all dates after the event
    future_dates = df.index[df.index > event_date]
    
    # Get the two earliest dates after the event
    upcoming_dates = future_dates.sort_values()[:range]
    
    if len(upcoming_dates) < range:
        return f"Not enough data after {event_date.date()} for ticker {ticker}"
    
    return df.loc[upcoming_dates]

def flatten_annotation(row):
    ann = row['annotation_json']
    
    # Copy all top-level keys except 'epidemiology'
    flat = {k: v for k, v in ann.items() if k != 'epidemiology'}
    
    # Handle epidemiology: sum of estimated_incidence across all conditions
    epidemiology = ann.get('epidemiology', {})
    total_incidence = sum(
        v.get('estimated_incidence', 0) 
        for v in epidemiology.values()
        if isinstance(v, dict)
    )
    flat['total_estimated_incidence'] = total_incidence
    
    return pd.Series(flat)


def get_volatility_and_price_change(financial_data, ticker, date_str):
    # Get the DataFrame for the specific ticker
    df = financial_data[ticker].copy()
    df.index = pd.to_datetime(df.index)

    # Convert the input string to a datetime object
    event_date = datetime.strptime(date_str, "%m_%d_%Y")
    
    # Define the range: from 60 days before the event up to 2 days before
    end_date = event_date - timedelta(days=2)
    start_date = event_date - timedelta(days=60)
    
    # Filter the DataFrame for the desired period
    mask = (df.index >= start_date) & (df.index <= end_date)
    df_period = df.loc[mask]

    if df_period.empty or df_period.shape[0] < 2:
        return f"Not enough data between {start_date.date()} and {end_date.date()} for ticker {ticker}"

    # Calculate daily returns
    df_period['daily_return'] = df_period['Close'].pct_change()

    # Calculate volatility (standard deviation of returns)
    volatility = df_period['daily_return'].std()

    # Calculate percent change from day -60 to day -2
    price_start = df_period['Close'].iloc[0]
    price_end = df_period['Close'].iloc[-1]

    percent_change = float(((price_end - price_start) / price_start))

    return {
        "volatility": round(volatility, 3),
        "percent_change": round(percent_change, 3)
        # "start_date": df_period.index[0].date(),
        # "end_date": df_period.index[-1].date(),
        # "df_period": df_period
    }



client = OpenAI(
    api_key="API_KEY",  # this is also the default, it can be omitted
)


In [ ]:
# Connect to the database
conn = sqlite3.connect("biotech_data.db")
cursor = conn.cursor()

# Enable returning rows as dictionaries (optional, but nice)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

cursor.execute('''
    SELECT 
        articles.id AS article_id,
        article_annotation.annotation_json,
        articles.date,
        company.ticker
    FROM article_annotation
    JOIN articles ON article_annotation.article_id = articles.id
    JOIN company ON articles.company_id = company.id
''')


# Fetch results
rows = cursor.fetchall()

# Convert to list of dictionaries, parsing JSON field
results = []
for row in rows:
    annotation = dict(row)
    annotation["annotation_json"] = json.loads(annotation["annotation_json"])  # Convert JSON string to dict
    results.append(annotation)

# Close the connection
conn.close()

# what does the ticker distribution look like?

# get tickers
tickers = [item['ticker'] for item in results]

# Count occurrences
ticker_counts = Counter(tickers)
ticker_counts


In [ ]:
# download historical data for all companies
import yfinance as yf
import pandas as pd
from tqdm import tqdm

financial_data = {}
start_date = "2014-01-01"
end_date = "2025-03-17"  # Adjust as needed

for ticker in tqdm(list(set(tickers))):
    # Fetch historical data
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    # Keep only Open, High, Low, Close columns
    stock_data = stock_data[['Open', 'High', 'Low', 'Close']]
    financial_data[ticker] = stock_data


# add the S&P 500 index as a reference
stock_data = yf.download("SPY", start=start_date, end=end_date)

# Keep only Open, High, Low, Close columns
stock_data = stock_data[['Open', 'High', 'Low', 'Close']]
financial_data['SPY'] = stock_data


# Assign small cap, mid cap, and large cap labels to each ticker
ticker_market_cap = {}

for ticker in tqdm(list(set(tickers))):
    try:
        info = yf.Ticker(ticker).info
        ticker_market_cap[ticker] = info['marketCap']
    except:
        print("error for:", ticker)
        pass

def classify_market_caps(market_caps_dict):
    cap_labels = {}
    for ticker, cap in market_caps_dict.items():
        if cap is None:
            cap_labels[ticker] = "Unknown"
        elif cap < 2e9:
            cap_labels[ticker] = "Small Cap"
        elif cap < 10e9:
            cap_labels[ticker] = "Mid Cap"
        else:
            cap_labels[ticker] = "Large Cap"
    return cap_labels


cap_classes = classify_market_caps(ticker_market_cap)


# how many tickers could we not fetch?
tickers_with_data = list(financial_data.keys())
count = 0
for ticker in list(set(tickers)):
    if ticker not in tickers_with_data:
        count = count + 1

# no missing data
print(count)


In [ ]:
# calculate data for all events
new_results = []
for result in tqdm(results):
    date = result['date']
    ticker = result['ticker']
    try:
        # Get 2-day pre-event close price
        price_data = get_two_days_before_event(financial_data, ticker, date)
        result['mean_price_minus_2_day_close'] = price_data['Close'][ticker].mean()

        # Extended days to include up to 60
        day_list = [1, 7, 14, 21, 30, 38, 45, 53, 60]

        # Get post-event close prices for each day in list
        for day in day_list:
            price_data = get_data_x_days_after_event(financial_data, ticker, date, day)
            result[f'mean_price_plus_{day}_day_close'] = price_data.loc[price_data.index.max(), 'Close'][ticker]

        # Calculate raw return ranges
        for day in day_list:
            result[f'{day}_day_range'] = (result[f'mean_price_plus_{day}_day_close'] - result['mean_price_minus_2_day_close']) / result['mean_price_minus_2_day_close']

        # SPY comparisons (market-adjusted)
        price_data = get_two_days_before_event(financial_data, "SPY", date)
        result['mean_spy_price_minus_2_day_close'] = price_data['Close']["SPY"].mean()

        for day in day_list:
            price_data = get_data_x_days_after_event(financial_data, "SPY", date, day)
            result[f'mean_spy_price_plus_{day}_day_close'] = price_data.loc[price_data.index.max(), 'Close']["SPY"]
            result[f'{day}_day_spy_range'] = (result[f'mean_spy_price_plus_{day}_day_close'] - result['mean_spy_price_minus_2_day_close']) / result['mean_spy_price_minus_2_day_close']
            result[f'adjusted_{day}_day_range'] = result[f'{day}_day_range'] - result[f'{day}_day_spy_range']

        new_results.append(result)

    except Exception as e:
        # Skip bad data points (e.g., delisted tickers, missing prices)
        pass

# convert to dataframe
df = pd.DataFrame(new_results)

# Extract the label and flatten the structure
df['label'] = df['annotation_json'].apply(lambda x: x.get('label', 'Unknown'))
df['adjusted_1_day_range'] = df['adjusted_1_day_range'].astype(float)

# Apply to your DataFrame
annotation_flat = df.apply(flatten_annotation, axis=1)

# Merge back into the original dataframe (or just use `annotation_flat` as new base)
df_expanded = pd.concat([df.drop(columns=['annotation_json']), annotation_flat], axis=1)

# add the cap labels
df_expanded["market_cap_label"] = df_expanded["ticker"].map(cap_classes)

df_expanded

In [ ]:
# filter the dataset down to reporting clinical trial results
data_for_annotation =  df_expanded[
    (df_expanded.event_type.isin(['Reporting Results & Data', 'Preliminary or Interim Results Shared']))
].copy()
data_for_annotation

In [ ]:
# fetch the article text and add it to the table
conn = sqlite3.connect("biotech_data.db")
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

# Query only article_id and article_text
cursor.execute('''
    SELECT 
        articles.id AS article_id,
        articles.text AS article_text
    FROM article_annotation
    JOIN articles ON article_annotation.article_id = articles.id
''')

# Fetch results and convert to DataFrame
rows = cursor.fetchall()
article_text_df = pd.DataFrame([dict(row) for row in rows])

# Drop duplicates in case of multiple annotations per article
article_text_df = article_text_df.drop_duplicates(subset='article_id')
data_for_annotation = data_for_annotation.merge(article_text_df, on='article_id', how='left')
data_for_annotation

In [ ]:
# Using LLMs to extract additional features out of it

new_annotations = []

for x in tqdm(range(0, len(data_for_annotation))):
    row = data_for_annotation.iloc[x]
    article_text = row['article_text']

    try:
        # Prompt 1 – Tone, narrative, and IR framing
        prompt_1 = f"""
        You are analyzing a biotech press release to extract narrative style, tone, promotional framing, and signals of forward progress. Return a JSON object with the following fields:

        {{
          "mentions_pipeline_expansion": <true / false>,
          "mentions_lack_of_competitors": <true / false>,
          "mentions_next_milestone": <true / false>,
          "mentions_specific_timeline": <true / false>,
          "timeline_confidence_score": <number from 1 (vague or speculative) to 10 (specific and confident)>,
          "mentions_upcoming_data_readout": <true / false>,
          "is_ir_tone_dominant": <true / false>,
          "ir_tone_score": <number from 1 (dry and technical, tailored for scientists) to 10 (promotional and tailored to investors)>,
          "uses_promotional_language": <true / false>,
          "uses_fomo_language": <true / false>,
          "narrative_style": "<Promotional / Technical / Strategic / Neutral>",
          "mentions_market_impact": <true / false>,
          "tone_score": <number from 1 (dry and technical) to 10 (optimistic and promotional)>,
          "forward_looking_statements": <true / false>,
          "hedging_score": <number from 1 (no hedging) to 10 (high hedging language)>,
          "bullish_sentiment_score": <number from 1 to 10>
        }}

        Press Release:
        \"\"\"{article_text}\"\"\"
        """
        result_1 = make_json_llm_call(client, prompt_1)

        # Prompt 2 – Clinical outcomes & real-world relevance
        prompt_2 = f"""
        Analyze this biotech press release to extract structured clinical outcomes and real-world relevance. Return the following JSON object:

        {{
          "mentions_statistical_significance": <true / false>,
          "efficacy_percent_improvement": <percentage or null>,
          "survival_rate_mentioned": <true / false>,
          "outcome_significance_score": <number from 1 (minor) to 10 (groundbreaking)>,
          "mentions_clinically_meaningful": <true / false>,
          "mentions_quality_of_life_improvement": <true / false>,
          "mentions_hospitalization_reduction": <true / false>,
          "mentions_symptom_relief": <true / false>,
          "mentions_real_world_relevance": <true / false>,
          "mentions_durable_response": <true / false>,
          "response_depth": "<Deep / Moderate / Minimal / Unknown>",
          "mentions_response_in_subgroups": <true / false>,
          "uses_vague_descriptors_only": <true / false>,
          "confidence_language_score": <number from 1 (highly uncertain, cautious tone) to 10 (very confident, assertive, and definitive tone)>,
          "mentions_adverse_events": <true / false>,
          "safety_profile_summary": "<Well-tolerated / Mixed / Concerning>"
        }}

        Press Release:
        \"\"\"{article_text}\"\"\"
        """
        result_2 = make_json_llm_call(client, prompt_2)

        # Prompt 3 – Strategic signals
        prompt_3 = f"""
        You are analyzing a biotech press release for signs of strategic shifts, urgency, and internal business pressures. Extract the following structured signals and return the following fields as JSON. If a field is unclear, return null and DO NOT GUESS.

        {{
          "mentions_funding_pressure": <true / false>,
          "mentions_employee_changes_or_hiring": <true / false>,
          "suggests_m_and_a_potential": <true / false>,
          "mentions_strategic_review_or_business_model_shift": <true / false>,
          "urgency_score": <1–10>,
          "mentions_cost_cutting": <true / false>,
          "mentions_pipeline_prioritization": <true / false>
        }}

        Press Release:
        \"\"\"{article_text}\"\"\"
        """
        result_3 = make_json_llm_call(client, prompt_3)

        # Prompt 4 – Commercial readiness
        prompt_4 = f"""
        You are analyzing a biotech press release for signs of commercial readiness and market entry planning. Extract the following fields as JSON. If a field is not applicable, return null.

        {{
          "mentions_pre_launch_preparations": <true / false>,
          "mentions_manufacturing_or_supply_chain": <true / false>,
          "mentions_distribution_partners": <true / false>,
          "mentions_sales_team_or_hiring": <true / false>,
          "mentions_market_access_or_reimbursement": <true / false>,
          "commercial_readiness_score": <1 (no evidence) to 10 (extensively discussed)>
        }}

        Press Release:
        \"\"\"{article_text}\"\"\"
        """
        result_4 = make_json_llm_call(client, prompt_4)

        # Prompt 5 – Audience targeting
        prompt_5 = f"""
        You are analyzing a biotech press release to understand who it is primarily written for and how it’s framed. Extract data from the attached press release and return JSON with the following structured information:

        {{
          "primary_audience": <"investor" | "scientific" | "media/general public" | "regulatory" | null>,
          "mentions_media_outlets_or_coverage": <true / false>,
          "mentions_social_or_patient_engagement": <true / false>,
          "includes_technical_figures_or_metrics": <true / false>,
          "mentions_target_stock_exchange_or_index": <true / false>,
          "language_alignment_with_regulators": <true / false>
        }}

        Press Release:
        \"\"\"{article_text}\"\"\"
        """
        result_5 = make_json_llm_call(client, prompt_5)

        # Prompt 6 – Trial design rigor
        prompt_6 = f"""
        You are analyzing a biotech press release for trial design features and study realism.Extract data from the attached press release and return JSON with the following structured information:

        {{
          "mentions_adaptive_trial_design": <true / false>,
          "mentions_placebo_or_comparator_arm": <true / false>,
          "mentions_real_world_evidence": <true / false>,
          "mentions_biomarker_selection": <true / false>,
          "mentions_global_or_multicenter_scope": <true / false>,
          "trial_design_rigor_score": <1 (minimal rigor) to 10 (robust and methodologically strong)>
        }}

        Press Release:
        \"\"\"{article_text}\"\"\"
        """
        result_6 = make_json_llm_call(client, prompt_6)

        # Prompt 7 – Novelty, credibility, confidence
        prompt_7 = f"""
        You are analyzing a biotech press release for subtle cues around market signaling, management confidence, novelty of the news, and credibility framing. Extract structured features as JSON using the definitions and value ranges provided.

        {{
          "announcement_novelty_score": <number from 1 (repeated or known) to 10 (entirely new and previously undisclosed)>,
          "mentions_previous_similar_announcements": <true / false>,
          "mentions_prior_failure_or_setback": <true / false>,
          "mentions_reversal_or_recovery": <true / false>,
          "mentions_internal_confidence_measures": <true / false>,
          "includes_third_party_validation": <true / false>,
          "mentions_patient_advocacy_group_or_foundation": <true / false>,
          "mentions_key_opinion_leader": <true / false>,
          "novel_mechanism_of_action": <true / false>,
          "competitive_differentiation_score": <number from 1 (none mentioned) to 10 (strongly differentiated vs competitors)>,
          "mentions_market_opportunity_size": <true / false>,
          "mentions_cost_effectiveness_or_pricing": <true / false>,
          "language_indicating_management_confidence": <true / false>,
          "mentions_risks_or_uncertainties": <true / false>,
          "risk_disclosure_score": <number from 1 (no mention of risk) to 10 (transparent, balanced discussion of risks)>,
          "overemphasized_positivity_score": <number from 1 (very neutral and factual) to 10 (heavily embellished or spin-heavy)>
        }}

        Press Release:
        \"\"\"{article_text}\"\"\"
        """
        result_7 = make_json_llm_call(client, prompt_7)

        # Merge all results
        combined_result = {
            **result_1,
            **result_2,
            **result_3,
            **result_4,
            **result_5,
            **result_6,
            **result_7,
        }

    except Exception as e:
        print(f"Annotation failed on row {x}: {e}")
        combined_result = {}

    new_annotations.append(combined_result)

# Final merge
annotations_df = pd.DataFrame(new_annotations)
data_for_annotation = pd.concat([data_for_annotation.reset_index(drop=True), annotations_df], axis=1)


In [ ]:
# save data to .csv files
data_for_annotation[data_for_annotation.result_type.isin(['Negative Results - Lack of Efficacy','Negative Results - Safety / Adverse Effect'])].to_csv("data/csv_files/negative_clinical_trial_results.csv")
data_for_annotation[data_for_annotation.result_type.isin(['Positive Results'])].to_csv("data/csv_files/positive_clinical_trial_results.csv")